![Trening_1](../img/oppdrag/oppdrag_1_bilde.png)


![Trening_1](../img/oppdrag/oppdrag_1_txt.png)


### Klient-Server kommunikasjon

![client_server](../img/assets/client_server.png)


Oppdraget vi skal løse er går ut på å sende og hente ut meldinger. Dette gjør datamaskiner ved hjelp av noe som heter Klient-server-kommunikasjon. Klient-server-kommunikasjon er en måte datamaskiner snakker sammen på.

Forestill deg at du er i et bibliotek og ønsker å låne en bok, dette blir ofte kalt en **Request**. Biblioteket har en bibliotekar (som er serveren) og deg selv (som er klienten). Du går til bibliotekaren og spør om boken du vil låne. Bibliotekaren sjekker om boken er tilgjengelig. Deretter gir bibliotekaren deg boken hvis den er tilgjengelig, dette blir ofte kalt en **Response**.

I dette eksemplet er du klienten, fordi du ber om noe (boken) fra bibliotekaren (serveren). Bibliotekaren har informasjonen (bokens tilgjengelighet) og svarer på forespørselen din. Enten ved å gi deg boken, eller ved å si at den ikke er tilgjengelig.

* **klient** er den som spør om informasjon. For eksempel en app eller nettleser
* **server** er datamaskinen som leverer informasjonen 

For å gjøre dette i Python må vi bruke `requests` pakken. Siden vi er interessert i å hente informasjon fra serveren legger vi til `get` eller hent på norsk. Da instruerer vi Python til å bruke en request til å hente informasjon. 

Serveren vi bruker for å lagre meldinger befinner seg på denne adressen:

`https://tenk-server.fly.dev/`

Vi til adressen i forespørselen vår, slik for å kunne få meldingene som befinner seg på serveren.

Dette har vi gjort i koden under, kjør blokken for å se hva som skjer

In [ ]:
import requests
server_adresse = "https://tenk-server.fly.dev/"

response = requests.get(server_adresse)
print(response)

Godt jobba! Du har fått det første svaret ditt fra serveren.

Kopier `https://tenk-server.fly.dev` inn i nettleseren din, ser du de samme meldingene der?


Men hva betyr egentlig `<Response [200]>`??

Det finnes en million måter å spørre en server om informasjon på, derfor har noen smarte mennesker laget en felles protokoll slik at alle gjør det på samme måte. Denne protokollen heter **HTTP**, kort for hypertekstoverføringsprotokoll. **HTTP** er den mest populære måten å utveklse informasjon på over internett. 

En viktig del av protokollen er de ulike kodene som serveren svarer med for å si noe om den har informasjonen vi spør om eller ikke. Under finner du de mest populære kodene.

| Statuskode | Beskrivelse                              |
|------------|------------------------------------------|
| 200        | OK - Forespørselen var vellykket         |
| 400        | Forespørselen din har ugyldig format     |
| 404        | Ressursen du spør etter finnes ikke      |
| 500        | Det er noe galt med serveren             |



#### Send en melding med HTTP
Forrige request vi sendte fikk vi en 200 response som sa at forespørselen vår var vellykket. Men vi så ikke noe informasjon, det er fordi denne er lagret i en [dictionary](oppdrag/trening.ipynb). Under viser vi ett eksempel på hvordan man kan skrive ut informasjonen som serveren har gitt oss

In [ ]:
import requests
server_adresse = "https://tenk-server.fly.dev/"

dictionary_med_meldinger = requests.get(server_adresse).json()
for melding in dictionary_med_meldinger:
    print(melding)


Nå som vi klarer å hente ut informasjon med `get` kommandoen er det på tide å sende informasjon til serveren.
Dette gjøres med `post` kommandoen. 


Når vi bruker post i requesten kan vi se for oss at vi sender med ett brev til serveren som innholder informasjon. Dette kommer vi til å omtale som data. Data som serveren kan lese må i ett [dictionary](oppdrag/trening.ipynb) format, dette lærte vi om i treningen.


Test dette ut med eksempelet under, her sender vi informasjon til serveren

Under bruker vi igjen `requests` pakken, men denne gangen med `post` kommandoen. `post` kommandoen forventer at vi gir den en server adresse og en datapakke som inneholder informasjon i ett dictionary format. 

In [ ]:
import requests

server_adresse = "https://tenk-server.fly.dev/"

navn = "Python"
beskjed = "Notebook" 

data = {
    "name": navn,
    "text": beskjed
}

response = requests.post(server_adresse, data=data)
print("Melding sendt!")


Kan du se meldingen din på skjermen? Det betyr at serveren har mottat og lagret denne. Hvis du ikke kunne sett meldingen på skjermen, hvordan kunne du vist om serveren mottok meldingen din?





 **Hint** statuskoder

#### Chat client
Vi ønsker å fange opp alle meldinger som blir sendt til den nye serveren, slik at vi kan dekode hemmlige meldinger.

I de tidligere cellene spurte vi hele tiden om informasjon fra serveren, dette kan være litt tungvindt når vi vet at vi kontinuerlig får inn ny informasjon fra klassen. Vi skal nå benytte oss av en teknologi som heter **web-sockets**. 

Metoden vi brukte tidligere med **HTTP klienten** kan sammenlignes med å sende brev, og vente på svar. Mens web-sockets kan sammenlignes med en walkie-talkie. Hvis noen snakker i sin walkie-talkie hører du det umiddelbart i din, og kan svare tilbake.

Vi skal nå lage en chat-client som får inn alle meldingene i sanntid ved hjelp av **web-sockets**. Kjør koden i cellene under for å teste det ut

In [ ]:
import socketio
sio = socketio.Client()

server_adresse = "https://tenk-server.fly.dev/"

@sio.on('messages')
def lytt_etter_melding(data):
    print("Vi får data fra serveren!")
    for melding in data:
        print(melding)
        
@sio.event
def koble_til():
    print("Jeg er koblet til!")

    
sio.connect(server_adresse)
sio.emit('newMessage', {'name': "hei", 'text': "test"})

: 

Vi kan og sende meldinger, akkurat som i walkie-talkie eksempelet. Her vil du se at de dukker opp på skjermen og i din egen notebook. I tillegg vil du kunne lese alle de andre meldingene som kommer inn.

In [ ]:
import socketio

navn = "FYLL INN NAVN"

sioClient = socketio.Client()
server_adresse = "https://tenk-server.fly.dev/"

@sioClient.event
def connect():
    print("Jeg er koblet til!")

sioClient.connect(server_adresse)
while True:
    input_fra_bruker = input("Skriv inn en melding (skriv exit for å avslutte): ")
    if input_fra_bruker.lower() == "exit":
        break
    else:
        sioClient.emit('newMessage', {'name': navn, 'text': input_fra_bruker})
sioClient.disconnect()
sio.disconnect()

### Hvis du kan se meldingen din kan du gå videre til [oppdrag_2](oppdrag_2.ipynb)

![Trening_1](../img/oppdrag/godt_jobba.png)
